# 02807 - Week 8 Exercises:  Getting started with Spark


# Learning objectives:

* Getting hands-on experience manipulating DataFrames with built-in Spark functions
* Distinguishing actions and transformations
* Writing your own UDFs


# Readings:


* [*Learning Spark*, Chapters 1-3](https://pages.databricks.com/rs/094-YMS-629/images/LearningSpark2.0.pdf). A nicely structured and detailed introduction to Spark.
* [A Neanderthal’s Guide to Apache Spark in Python](https://towardsdatascience.com/a-neanderthals-guide-to-apache-spark-in-python-9ef1f156d427). A brief, fun and gentle introduction to Spark for complete beginners.


# Setup

You'll need to get pyspark and make some imports. The following cells will get you started.

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()

# Exercise 1: age bracketing for the Titanic Dataset

In this exercise you should use Spark to count the number of Titanic passengers in different age brackets. More specifically, you need to count the number of people age 0 to 9, 10 to 19, and so on.


## Loading the data

Load the Titanic data used in the lecture slides into a Spark dataframe (use schema inference).

In [ ]:
# Your code goes here

## Cleaning the data

Remove the rows that do not have an age 


In [ ]:
# Your code goes here

## Adding age brackets 

Create a new column with a value that identifies the bracket that passengers are in

In [ ]:
# Your code goes here

## Age bracket counts

Create a Spark dataframe with the sum of passengers in each bracket

In [ ]:
# Your code goes here

# Exercise 2: understanding actions and transformations

For each of the following Spark operations, decide if they are transformations or actions. If they are transformations, determine if they are wide or narrow.

* ``select()``
* `groupBy()`
* `filter()`
* `where()`
* `count()`
* `show()`
* `agg()`
* `write()`

*Your answers go here*

# Exercise 3: exploratory data analysis for the Chicago crime dataset


The Chicago Crime dataset contains a summary of the reported crimes occurred in the City of Chicago from 2001 to 2017. 

It is a fairly large dataset. You'll work with a sample of it. Execute the following cells to load it into a dataframe.



In [ ]:
# to get the full dataset, run: !wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
!wget https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv

In [ ]:
!ls -1
!mv rows.csv\?accessType\=DOWNLOAD reported-crimes.csv
!ls -l

In [ ]:
rc = spark.read.csv('reported-crimes.csv',header=True)
rc.show()

Let's do some EDA. Answer the following questions.

**What percentage of reported crimes resulted in an arrest?**

In [ ]:
# Your code goes here

**What are the top 3 locations for reported crimes?**

In [ ]:
# Your code goes here

**What is the most common primary type of crime in district 022?**


In [ ]:
# Your code goes here